In [1]:
from requests.auth import HTTPBasicAuth
import requests
import pandas as pd
import os
import time
from datetime import datetime

In [ ]:
# list of terms that needs to be searched  
searchterms = ["downsize", "downsizing" , "layoff", "lost job", "lost their job", "let go", "workforce reduction", 
               "jobs eliminated" , "position eliminated"]

#configure
CONTENT = 'News' # Media Coverage, Business Press Release

# Read data in
layoffs = pd.read_xlsx("./Mannor Company Layoffs.xlsx", sheet_name='Key Fields')
# filter that will be applied in the URL
FILTER  = "SearchType eq LexisNexis.ServicesApi.SearchType'Boolean' and \
GroupDuplicates eq LexisNexis.ServicesApi.GroupDuplicates'ModerateSimilarity' and \
Language eq LexisNexis.ServicesApi.Language'English' and \
(PublicationType eq 'TmV3c3BhcGVycw' or PublicationType eq 'TWFnYXppbmVzICYgSm91cm5hbHM') and \
Location ne 'dXJuOmtybTpwdmktRjQ3RUVCRjM1NkVENDk5RThBOEREMkJGNDI5MjM1OUM'"

DEBUG   = False

def get_token( client_id, secret ) :
    """Gets Authorizaton token to use in other requests."""
    auth_url  = 'https://auth-api.lexisnexis.com/oauth/v2/token'
    payload   = ( 'grant_type=client_credentials&scope=http%3a%2f%2f' 'oauth.lexisnexis.com%2fall' )
    headers   = { 'Content-Type': 'application/x-www-form-urlencoded' }
    r         = requests.post( auth_url, auth=HTTPBasicAuth( client_id, secret ), headers=headers, data=payload )
    json_data = r.json()
    return json_data[ 'access_token' ]

def build_header( token ) : 
    """Builds the headers part of the request to Web Services API."""
    headers = { 'Accept': 'application/json;odata.metadata=minimal', 'Connection': 'Keep-Alive', 'Host': 'services-api.lexisnexis.com' }
    headers[ 'Authorization' ] = 'Bearer ' + token
    return headers

def build_url( content=CONTENT, query='', skip=0, expand='Document', top=50, filter=None ) :
	"""Builds the URL part of the request to Web Services API."""
	# check for filter
	if filter != None : api_url = ('https://services-api.lexisnexis.com/v1/' + content + '?$expand=' + expand + '&$search=' + query + '&$skip=' + str(skip) + '&$top=1&$filter=' + filter )
	else : api_url = ('https://services-api.lexisnexis.com/v1/' + content + '?$expand=' + expand + '&$search=' + query + '&$skip=0' + '&$top=1')
	return api_url

# initialize
CLIENTID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
SECRET   = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
headers = build_header(get_token( CLIENTID, SECRET ) )

layoffs = pd.read_excel("./Mannor Company Layoffs.xlsx", sheet_name='Key Fields')
data = layoffs[['conm', 'fyear']]

for index, row in data.iterrows():
    company = row['conm'] # How to get the company into the query?
    year = row['fyear']
    # How can we limit our results to just the first 10-ish results? 
    # Are we limited on articles we pull or times we pull?
    
    querydates = ['date>=01/01/'+str(year)+' AND date<=01/31/'+str(year)+'',
                'date>=02/01/'+str(year)+' AND date<=02/29/'+str(year)+'',
                'date>=03/01/'+str(year)+' AND date<=03/31/'+str(year)+'',
                'date>=04/01/'+str(year)+' AND date<=04/30/'+str(year)+'',
                'date>=05/01/'+str(year)+' AND date<=05/31/'+str(year)+'',
                'date>=06/01/'+str(year)+' AND date<=06/30/'+str(year)+'',
                'date>=07/01/'+str(year)+' AND date<=07/31/'+str(year)+'',
                'date>=08/01/'+str(year)+' AND date<=08/31/'+str(year)+'',
                'date>=09/01/'+str(year)+' AND date<=09/30/'+str(year)+'',
                'date>=10/01/'+str(year)+' AND date<=10/31/'+str(year)+'',
                'date>=11/01/'+str(year)+' AND date<=11/30/'+str(year)+'',
                'date>=12/01/'+str(year)+' AND date<=12/31/'+str(year)+'']

    ncalls = 0 
    for searchterm in searchterms:
        if not os.path.exists(os.getcwd()+'/'+searchterm):
            os.makedirs(os.getcwd()+'/'+searchterm)
        else:
            print(os.getcwd()+'/'+searchterm, 'already exists!')
            # continue

        for i in range(1,13):
            monthnum = i # monthnum serves as output file name
            querydate = querydates[(monthnum-1)]
            
            QUERY_hit    = '"' + searchterm + '" ' + querydate
            QUERY_tot =  querydate
                
            # Filter is set to filter=None here. Change to filter=filter to use the filter specified above
            url_hit = build_url(content=CONTENT, query=QUERY_hit, skip=0, expand='Document', top=1, filter=FILTER )  
            url_tot = build_url(content=CONTENT, query=QUERY_tot, skip=0, top=1, filter=FILTER )  
            
            # sole call to retrieve total
            response_tot = requests.get(url_tot, headers=headers)
            time.sleep(87) #pause to meet 1000 hits/24hr limit
            ncalls = ncalls+1
            try:
                j_tot = response_tot.json()
                n_total = j_tot['@odata.count']
            except:
                print("first fail"+str(datetime.now()))
                print(searchterm, '--> invalid response reached:')
                print(j_tot)
                print('ncalls:',ncalls)

                # before quitting we want to get a new token seems like after 24h you need to get a new token to continue execution
                headers = build_header(get_token(CLIENTID, SECRET ) )
                print("getting new token...")
                # sole call to retrieve total
                response_tot = requests.get(url_tot, headers=headers)
                time.sleep(87) #pause to meet 1000 hits/24hr limit
                ncalls = ncalls+1
                try:
                    j_tot = response_tot.json()
                    n_total = j_tot['@odata.count']
                except:
                    print("second fail"+str(datetime.now()))
                    print(searchterm, '--> invalid response reached with renewed token:')
                    print(j_tot)
                    print('ncalls:',ncalls)
                    break
            
            df_giant = pd.DataFrame()
        
            while True:
                print(datetime.now())
                response_hit = requests.get(url_hit, headers=headers)
                print("wait...")
                time.sleep(87) #pause to meet 1000 hits/24hr limit
                #print(datetime.now())
                ncalls = ncalls+1
                print('ncalls:',ncalls)
                try:
                    j_hit = response_hit.json()
                except:
                    break
                else:
                    try:
                        df = pd.json_normalize(j_hit['value'][0])
                    except:
                        pass
                    else:
                        df_giant = df_giant.append(df)

                        n_hits = j_hit['@odata.count']
                    
                        df_giant['n_hits'] = n_hits
                        df_giant['n_total'] = n_total    
                        if not os.path.isfile(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv'):
                            df_giant.to_csv(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv', header=True, encoding='utf-8-sig', index=False)
                        else: # else it exists so append without writing the header
                            df_giant.to_csv(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv', mode='a', header=False, encoding='utf-8-sig', index=False)
                    
                    try:
                        url_hit = j_hit['@odata.nextLink']
                    except:
                        break
            del response_tot
            del response_hit
            df_giant.reset_index(inplace=True, drop=True)
            if not os.path.isfile(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv'):
                df_giant.to_csv(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv', header=True, encoding='utf-8-sig', index=False)
            else: # else it exists so append without writing the header
                df_giant.to_csv(searchterm + '/' + str(year) + '_' + str(monthnum) + '.csv', mode='a', header=False, encoding='utf-8-sig', index=False)

In [6]:
layoffs = pd.read_excel("./LexisNexis/Mannor Company Layoffs.xlsx", sheet_name='Key Fields')
data = layoffs[['conm', 'fyear']]
for index, row in data.iterrows():
    company = row['conm']
    year = row['fyear']
    print(company, year)

AAR CORP 2014
AAR CORP 2020
AMERICAN AIRLINES GROUP INC 2008
AMERICAN AIRLINES GROUP INC 2009
AMERICAN AIRLINES GROUP INC 2010
AMERICAN AIRLINES GROUP INC 2012
AMERICAN AIRLINES GROUP INC 2020
ABBOTT LABORATORIES 2013
ABBOTT LABORATORIES 2015
ADVANCED MICRO DEVICES 2008
ADVANCED MICRO DEVICES 2009
ADVANCED MICRO DEVICES 2010
ADVANCED MICRO DEVICES 2012
ADVANCED MICRO DEVICES 2014
ADVANCED MICRO DEVICES 2015
ADVANCED MICRO DEVICES 2016
AETNA INC 2010
AETNA INC 2011
AETNA INC 2016
AETNA INC 2017
AIR PRODUCTS & CHEMICALS INC 2008
AIR PRODUCTS & CHEMICALS INC 2009
AIR PRODUCTS & CHEMICALS INC 2010
AIR PRODUCTS & CHEMICALS INC 2015
AIR PRODUCTS & CHEMICALS INC 2016
AIR PRODUCTS & CHEMICALS INC 2017
ALASKA AIR GROUP INC 2008
ALASKA AIR GROUP INC 2009
ALASKA AIR GROUP INC 2020
MATSON INC 2012
HONEYWELL INTERNATIONAL INC 2009
HONEYWELL INTERNATIONAL INC 2013
HONEYWELL INTERNATIONAL INC 2014
HONEYWELL INTERNATIONAL INC 2018
HONEYWELL INTERNATIONAL INC 2019
HONEYWELL INTERNATIONAL INC 2020
HONEY

NIKE INC  -CL B 2019
NIKE INC  -CL B 2020
NORDSON CORP 2021
NORDSTROM INC 2008
NORDSTROM INC 2018
NORDSTROM INC 2019
NORDSTROM INC 2020
NORFOLK SOUTHERN CORP 2009
NORFOLK SOUTHERN CORP 2013
NORFOLK SOUTHERN CORP 2014
NORFOLK SOUTHERN CORP 2016
NORFOLK SOUTHERN CORP 2017
NORFOLK SOUTHERN CORP 2019
NORFOLK SOUTHERN CORP 2020
NORFOLK SOUTHERN CORP 2021
NISOURCE INC 2015
NISOURCE INC 2020
NORTHROP GRUMMAN CORP 2009
NORTHROP GRUMMAN CORP 2010
NORTHROP GRUMMAN CORP 2011
NORTHROP GRUMMAN CORP 2012
NORTHROP GRUMMAN CORP 2013
NORTHROP GRUMMAN CORP 2014
NORTHROP GRUMMAN CORP 2021
TEREX CORP 2008
TEREX CORP 2009
TEREX CORP 2012
TEREX CORP 2013
TEREX CORP 2016
TEREX CORP 2017
TEREX CORP 2019
TEREX CORP 2020
WELLS FARGO & CO 2009
WELLS FARGO & CO 2011
WELLS FARGO & CO 2013
WELLS FARGO & CO 2017
WELLS FARGO & CO 2018
WELLS FARGO & CO 2021
NUCOR CORP 2009
OCCIDENTAL PETROLEUM CORP 2014
OCCIDENTAL PETROLEUM CORP 2015
OCCIDENTAL PETROLEUM CORP 2020
OCEANEERING INTERNATIONAL 2015
OCEANEERING INTERNATION

NEWS CORP 2016
NEWS CORP 2020
MALLINCKRODT PLC 2016
MALLINCKRODT PLC 2017
SCIENCE APPLICATIONS INTL CP 2013
G-III APPAREL GROUP LTD 2018
G-III APPAREL GROUP LTD 2020
KNOWLES CORP 2015
KNOWLES CORP 2016
KNOWLES CORP 2020
GANNETT CO INC 2020
GANNETT CO INC 2021
VERITIV CORP 2019
VERITIV CORP 2020
SCOTTS MIRACLE-GRO CO 2016
RAYONIER ADVANCED MATERIALS 2021
TESCO CORP 2009
TESCO CORP 2015
CISCO SYSTEMS INC 2009
CISCO SYSTEMS INC 2012
HOLOGIC INC 2013
HOLOGIC INC 2020
TETRA TECHNOLOGIES INC/DE 2015
TETRA TECHNOLOGIES INC/DE 2016
TETRA TECHNOLOGIES INC/DE 2020
TETRA TECHNOLOGIES INC/DE 2021
AVANOS MEDICAL INC 2018
AVANOS MEDICAL INC 2021
GRANITE CONSTRUCTION INC 2008
GRANITE CONSTRUCTION INC 2009
GRANITE CONSTRUCTION INC 2014
GRANITE CONSTRUCTION INC 2021
B/E AEROSPACE INC 2009
B/E AEROSPACE INC 2014
KEMPER CORP/DE 2012
KEMPER CORP/DE 2014
CDK GLOBAL INC 2021
CITIZENS FINANCIAL GROUP INC 2015
INTL BANCSHARES CORP 2020
TRUEBLUE INC 2008
TRUEBLUE INC 2020
HCA HEALTHCARE INC 2009
HCA HEALTHCARE

AGILENT TECHNOLOGIES INC 2009
AGILENT TECHNOLOGIES INC 2015
TELEDYNE TECHNOLOGIES INC 2009
TELEDYNE TECHNOLOGIES INC 2015
TELEDYNE TECHNOLOGIES INC 2020
PACKAGING CORP OF AMERICA 2015
EDGEWELL PERSONAL CARE CO 2015
KRISPY KREME INC 2011
METLIFE INC 2009
METLIFE INC 2012
METLIFE INC 2016
METLIFE INC 2017
METLIFE INC 2018
METLIFE INC 2020
METLIFE INC 2021
FRONTIER COMMUNIC PARENT INC 2012
FRONTIER COMMUNIC PARENT INC 2013
FRONTIER COMMUNIC PARENT INC 2017
FRONTIER COMMUNIC PARENT INC 2018
FRONTIER COMMUNIC PARENT INC 2019
FRONTIER COMMUNIC PARENT INC 2020
FRONTIER COMMUNIC PARENT INC 2021
NRG ENERGY INC 2013
NRG ENERGY INC 2016
NRG ENERGY INC 2017
NRG ENERGY INC 2018
BASIC ENERGY SERVICES INC 2015
BASIC ENERGY SERVICES INC 2019
CHARLES RIVER LABS INTL INC 2009
MEDICINES CO (THE) 2016
MEDICINES CO (THE) 2017
MEDICINES CO (THE) 2018
MOODY'S CORP 2019
TTM TECHNOLOGIES INC 2009
TTM TECHNOLOGIES INC 2011
TTM TECHNOLOGIES INC 2013
TTM TECHNOLOGIES INC 2016
TTM TECHNOLOGIES INC 2018
TTM TECHNOL